In [1]:
import pandas as pd
import math
from IPython.display import display, HTML
import numpy as np

In [2]:
# Assign csv file to each variable

date_name = "2022-12-15"        # Sesuaikan dengan nama file, perhatikan nama filenya dan penulisan code di bagian ini 

df_odom = []
df_imu = []
df_amcl = []
df_ekf = []

for i in range(5):
    file_csv_odom = "bag/" + date_name + "-" + str(i) + "-" + "odom" + ".csv"       # Sesuai nama file
    df_odom.append(pd.read_csv(file_csv_odom))
    df_odom[i] = df_odom[i][[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"]]   # Jenis data yang dipakai  

    file_csv_imu = "bag/" + date_name + "-" + str(i) + "-" + "odometry-imu" + ".csv"
    df_imu.append(pd.read_csv(file_csv_imu))
    df_imu[i] = df_imu[i][[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"]]
    
    file_csv_amcl = "bag/" + date_name + "-" + str(i) + "-" + "amcl_pose" + ".csv"
    df_amcl.append(pd.read_csv(file_csv_amcl))
    df_amcl[i] = df_amcl[i][[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"]]

    file_csv_ekf = "bag/" + date_name + "-" + str(i) + "-" + "odometry-filtered" + ".csv"
    df_ekf.append(pd.read_csv(file_csv_ekf))
    df_ekf[i] = df_ekf[i][[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"]]

In [3]:
# check data length of each variable

for i in range(5):
    print("{}) odom: {} -- ekf imu: {} -- amcl: {} -- ekf: {}".format(i+1, len(df_odom[i]), len(df_imu[i]), len(df_amcl[i]), len(df_ekf[i])))

1) odom: 4875 -- ekf imu: 1829 -- amcl: 67 -- ekf: 1829
2) odom: 4854 -- ekf imu: 1819 -- amcl: 69 -- ekf: 1820
3) odom: 6612 -- ekf imu: 2480 -- amcl: 69 -- ekf: 2479
4) odom: 6498 -- ekf imu: 2436 -- amcl: 72 -- ekf: 2436
5) odom: 6635 -- ekf imu: 2488 -- amcl: 70 -- ekf: 2488


In [4]:
# check NaN value in each variable

for i in range(5):
    print("{}) NaN odom: {} -- NaN ekf imu: {} -- NaN amcl: {} -- NaN ekf: {}".format(i+1, str(df_odom[i].isna().sum().sum()), str(df_imu[i].isna().sum().sum()), str(df_amcl[i].isna().sum().sum()), str(df_ekf[i].isna().sum().sum())))

1) NaN odom: 0 -- NaN ekf imu: 0 -- NaN amcl: 0 -- NaN ekf: 0
2) NaN odom: 0 -- NaN ekf imu: 0 -- NaN amcl: 0 -- NaN ekf: 0
3) NaN odom: 0 -- NaN ekf imu: 0 -- NaN amcl: 0 -- NaN ekf: 0
4) NaN odom: 0 -- NaN ekf imu: 0 -- NaN amcl: 0 -- NaN ekf: 0
5) NaN odom: 0 -- NaN ekf imu: 0 -- NaN amcl: 0 -- NaN ekf: 0


In [5]:
# Uncomment this if you want to show your data
# display(HTML(df_amcl[3].to_html()))

In [6]:
df_odom_clean = []
df_imu_clean = []
df_amcl_clean = []
df_ekf_clean = []

for i in range(5):
    df_odom_clean.append(pd.DataFrame(
        columns=[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"]))

    df_imu_clean.append(pd.DataFrame(
        columns=[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"]))

    df_amcl_clean.append(pd.DataFrame(
        columns=[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"]))

    df_ekf_clean.append(pd.DataFrame(
        columns=[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"]))

In [7]:
prev = 0
for l in range(5):
    for j, i in enumerate(df_amcl[l][".header.stamp.secs"]):
        if prev == i:
            continue
        df_amcl_clean_ = df_amcl[l].iloc[j].to_frame().T
        df_ekf_clean_ = (df_ekf[l][df_ekf[l][".header.stamp.secs"] == i])
        df_odom_clean_ = (df_odom[l][df_odom[l][".header.stamp.secs"] == i])
        df_imu_clean_ = (df_imu[l][df_imu[l][".header.stamp.secs"] == i])
        try:
            df_ekf_clean_ = df_ekf_clean_.reset_index().iloc[0].to_frame().T
            df_odom_clean_ = df_odom_clean_.reset_index().iloc[0].to_frame().T
            df_imu_clean_ = df_imu_clean_.reset_index().iloc[0].to_frame().T
        except:
            df_amcl_clean_ = pd.DataFrame(
                columns=[".header.stamp.secs", ".pose.pose.position.x", ".pose.pose.position.y"])
            pass
        prev = i
        df_ekf_clean[l] = pd.concat(
            [df_ekf_clean[l], df_ekf_clean_], ignore_index=True, axis=0)
        df_amcl_clean[l] = pd.concat(
            [df_amcl_clean[l], df_amcl_clean_], ignore_index=True, axis=0)
        df_odom_clean[l] = pd.concat(
            [df_odom_clean[l], df_odom_clean_], ignore_index=True, axis=0)
        df_imu_clean[l] = pd.concat(
            [df_imu_clean[l], df_imu_clean_], ignore_index=True, axis=0)

In [8]:
for i in range(5):
    print("{}) odom: {} -- ekf imu: {} -- amcl: {} -- ekf: {}".format(i+1,
          len(df_odom_clean[i]), len(df_imu_clean[i]), len(df_amcl_clean[i]), len(df_ekf_clean[i])))

1) odom: 50 -- ekf imu: 50 -- amcl: 50 -- ekf: 50
2) odom: 50 -- ekf imu: 50 -- amcl: 50 -- ekf: 50
3) odom: 52 -- ekf imu: 52 -- amcl: 52 -- ekf: 52
4) odom: 53 -- ekf imu: 53 -- amcl: 53 -- ekf: 53
5) odom: 56 -- ekf imu: 56 -- amcl: 56 -- ekf: 56


In [9]:
min_amcl = len(df_amcl_clean[0])
index_amcl = df_amcl_clean.index(df_amcl_clean[0])
 
for i in range(len(df_amcl_clean)):
    if (len(df_amcl_clean[i])) < min_amcl:
        min_amcl = len(df_amcl_clean[i])
        index_amcl = df_amcl_clean.index(df_amcl_clean[i])

print("The smallest len in the list is {} in {}".format(min_amcl, index_amcl))

The smallest len in the list is 50 in 0


In [10]:
while (True):
    if (min_amcl < len(df_amcl_clean[0])):
        df_amcl_clean[0].drop(df_amcl_clean[0].index[-1], inplace=True)
        df_odom_clean[0].drop(df_odom_clean[0].index[-1], inplace=True)
        df_imu_clean[0].drop(df_imu_clean[0].index[-1], inplace=True)
        df_ekf_clean[0].drop(df_ekf_clean[0].index[-1], inplace=True)

    if (min_amcl < len(df_amcl_clean[1])):
        df_amcl_clean[1].drop(df_amcl_clean[1].index[-1], inplace=True)
        df_odom_clean[1].drop(df_odom_clean[1].index[-1], inplace=True)
        df_imu_clean[1].drop(df_imu_clean[1].index[-1], inplace=True)
        df_ekf_clean[1].drop(df_ekf_clean[1].index[-1], inplace=True)

    if (min_amcl < len(df_amcl_clean[2])):
        df_amcl_clean[2].drop(df_amcl_clean[2].index[-1], inplace=True)
        df_odom_clean[2].drop(df_odom_clean[2].index[-1], inplace=True)
        df_imu_clean[2].drop(df_imu_clean[2].index[-1], inplace=True)
        df_ekf_clean[2].drop(df_ekf_clean[2].index[-1], inplace=True)

    if (min_amcl < len(df_amcl_clean[3])):
        df_amcl_clean[3].drop(df_amcl_clean[3].index[-1], inplace=True)
        df_odom_clean[3].drop(df_odom_clean[3].index[-1], inplace=True)
        df_imu_clean[3].drop(df_imu_clean[3].index[-1], inplace=True)
        df_ekf_clean[3].drop(df_ekf_clean[3].index[-1], inplace=True)

    if (min_amcl < len(df_amcl_clean[4])):
        df_amcl_clean[4].drop(df_amcl_clean[4].index[-1], inplace=True)
        df_odom_clean[4].drop(df_odom_clean[4].index[-1], inplace=True)
        df_imu_clean[4].drop(df_imu_clean[4].index[-1], inplace=True)
        df_ekf_clean[4].drop(df_ekf_clean[4].index[-1], inplace=True)

    if (min_amcl == len(df_amcl_clean[1]) == len(df_amcl_clean[2]) == len(df_amcl_clean[3]) == len(df_amcl_clean[4])):
        break

In [11]:
for i in range(5):
    print("{}) odom: {} -- ekf imu: {} -- amcl: {} -- ekf: {}".format(i+1,
          len(df_odom_clean[i]), len(df_imu_clean[i]), len(df_amcl_clean[i]), len(df_ekf_clean[i])))


1) odom: 50 -- ekf imu: 50 -- amcl: 50 -- ekf: 50
2) odom: 50 -- ekf imu: 50 -- amcl: 50 -- ekf: 50
3) odom: 50 -- ekf imu: 50 -- amcl: 50 -- ekf: 50
4) odom: 50 -- ekf imu: 50 -- amcl: 50 -- ekf: 50
5) odom: 50 -- ekf imu: 50 -- amcl: 50 -- ekf: 50


In [12]:
for i in range(5):
    df_odom_clean[i][".header.stamp.secs"] = df_odom_clean[i][".header.stamp.secs"].astype(int)
    df_ekf_clean[i][".header.stamp.secs"] = df_ekf_clean[i][".header.stamp.secs"].astype(int)
    df_amcl_clean[i][".header.stamp.secs"] = df_amcl_clean[i][".header.stamp.secs"].astype(int)
    df_imu_clean[i][".header.stamp.secs"] = df_imu_clean[i][".header.stamp.secs"].astype(int)

In [13]:
for i in range(5):
    df_odom_clean[i].drop(labels='index', axis=1, inplace=True)
    df_ekf_clean[i].drop(labels='index', axis=1, inplace=True)
    df_imu_clean[i].drop(labels='index', axis=1, inplace=True)
    try:
        df_amcl_clean[i].drop(labels='index', axis=1, inplace=True)
    except:
        pass

In [36]:
# display(
#     HTML(df_amcl_clean[0].to_html()), 
#     HTML(df_odom_clean[0].to_html()),
#     HTML(df_ekf_clean[0].to_html())
#     )

In [15]:
# Ground truth lintasan persegi
x_max = 5.6
y_max = -3.2
kel = abs((2*x_max)) + abs((2*y_max))
# print(kel)

pjg_data = len(df_ekf_clean[4])
new_time = kel/pjg_data
# print(new_time)

x_truth = []
y_truth = []
point_x = 0
point_y = 0
while (point_x < x_max):
    x_truth.append(point_x)
    y_truth.append(point_y)
    point_x += new_time

x_truth[-1] = 5.5
y_truth[-1] = 0

x_max = x_truth[-1]
point_x = x_max


while (y_max < point_y):
    point_y -= new_time
    if point_y < y_max:
        break
    y_truth.append(point_y)
    x_truth.append(x_max)

y_max = y_truth[-1]
point_y = y_max


while (0 < point_x):
    point_x -= new_time
    if point_x < 0:
        break
    x_truth.append(point_x)
    y_truth.append(point_y)

x_truth[-1] = 0
y_truth[-1] = -3.168

x_max = x_truth[-1]
point_x = x_max


while (point_y < 0):
    point_y += new_time
    if 0 < point_y:
        break
    y_truth.append(point_y)
    x_truth.append(x_max)

x_truth.append(0)
y_truth.append(0)

# print(point_x, point_y)
# print(x_truth[-1], y_truth[-1])
print(len(x_truth), len(y_truth))

49 49


In [32]:
import plotly.graph_objects as go

# Transparan
layout = go.Layout(
    # paper_bgcolor='rgba(0,0,0,0)',
    # plot_bgcolor='rgba(0,0,0,0)',
    # showlegend=False,
    # autosize=False,
    # width=1000,
    # height=800,
    margin=go.layout.Margin(
        l=20,
        r=20,
        b=20,
        t=20,
        pad = 4
    )
)

fig = go.Figure(layout=layout)

# Add traces
for i in range(5):
    fig.add_trace(go.Scatter(x=df_odom_clean[i][".pose.pose.position.x"], y=df_odom_clean[i][".pose.pose.position.y"],
                        mode='lines+markers',
                        opacity=0,
                        # name='ODOMETER {}'.format(i+1),
                        showlegend=False
                        ))
    # fig.add_trace(go.Scatter(x=df_imu_clean[i][".pose.pose.position.x"], y=df_imu_clean[i][".pose.pose.position.y"],
    #                     mode='lines+markers',
    #                     name='EKF ODOMETER-IMU {}'.format(i+1)))
    fig.add_trace(go.Scatter(x=df_amcl_clean[i][".pose.pose.position.x"], y=df_amcl_clean[i][".pose.pose.position.y"],
                        mode='lines+markers',
                        name='AMCL {}'.format(i+1)))
    fig.add_trace(go.Scatter(x=df_ekf_clean[i][".pose.pose.position.x"], y=df_ekf_clean[i][".pose.pose.position.y"],
                        mode='lines+markers',
                        name='EKF {}'.format(i+1)))

# truth lintasan persegi
fig.add_trace(go.Scatter(x=x_truth, y=y_truth,
                    mode='lines+markers',
                    name='LINTASAN SEHARUSNYA'))

# truth 3 titik
# fig.add_trace(go.Scatter(x=x_three_truth, y=y_three_truth,
#                     mode='markers',
#                     name='GOAL',
#                     marker_size=20,
#                     marker_symbol='star'
#                     ))

# Remove grid, zero line, axis number
fig.update_xaxes(
    showgrid=False, 
    # zeroline=False, 
    # visible=False,
    zerolinecolor='red',
    gridcolor='LightPink',
    )
fig.update_yaxes(
    showgrid=False, 
    # zeroline=False, 
    # visible=False,
    zerolinecolor='red',
    gridcolor='LightPink',
    )

fig.update_layout(
    # font_family="Courier New",
    font_color="red",
    font_size=35,
    # title_font_family="Times New Roman",
    # title_font_color="red",
    legend = dict(font = dict(family = "Arial", size = 20, color = "black"))
)

fig.show()

In [17]:
NE_odom = []
NEk_odom = 0
ME_odom = []
NE_ME = []
MSE_odom = []
x_odom = []
y_odom = []

for l in range(5):
    NE_odom.append([])
    NE_ME.append([])
    x_odom.append(df_odom_clean[l][".pose.pose.position.x"])
    y_odom.append(df_odom_clean[l][".pose.pose.position.y"])

    for i,j in enumerate(x_truth):
        ex = x_odom[l][i]-j
        ey = y_odom[l][i]-y_truth[i]
        NEk = math.sqrt(ex**2+ey**2)
        NEk_odom = NEk
        NE_odom[l].append(NEk_odom)
    
    ME_odom.append(sum(NE_odom[l])/len(df_odom_clean[l]))
    
    for i,j in enumerate(NE_odom[l]):
        NE_ME[l].append((j-ME_odom[l])**2)

    MSE_odom.append(sum(NE_ME[l])/len(df_odom_clean[l]))

In [18]:
NE_imu = []
NEk_imu = 0
ME_imu = []
NE_ME = []
MSE_imu = []
x_imu = []
y_imu = []

for l in range(5):
    NE_imu.append([])
    NE_ME.append([])
    x_imu.append(df_imu_clean[l][".pose.pose.position.x"])
    y_imu.append(df_imu_clean[l][".pose.pose.position.y"])

    for i,j in enumerate(x_truth):
        ex = x_imu[l][i]-j
        ey = y_imu[l][i]-y_truth[i]
        NEk = math.sqrt(ex**2+ey**2)
        NEk_imu = NEk
        NE_imu[l].append(NEk_imu)
    
    ME_imu.append(sum(NE_imu[l])/len(df_imu_clean[l]))
    
    for i,j in enumerate(NE_imu[l]):
        NE_ME[l].append((j-ME_imu[l])**2)

    MSE_imu.append(sum(NE_ME[l])/len(df_imu_clean[l]))

In [19]:
NE_amcl = []
NEk_amcl = 0
ME_amcl = []
NE_ME = []
MSE_amcl = []
x_amcl = []
y_amcl = []

for l in range(5):
    NE_amcl.append([])
    NE_ME.append([])
    x_amcl.append(df_amcl_clean[l][".pose.pose.position.x"])
    y_amcl.append(df_amcl_clean[l][".pose.pose.position.y"])

    for i,j in enumerate(x_truth):
        ex = x_amcl[l][i]-j
        ey = y_amcl[l][i]-y_truth[i]
        NEk = math.sqrt(ex**2+ey**2)
        NEk_amcl = NEk
        NE_amcl[l].append(NEk_amcl)
    
    ME_amcl.append(sum(NE_amcl[l])/len(df_amcl_clean[l]))
    
    for i,j in enumerate(NE_amcl[l]):
        NE_ME[l].append((j-ME_amcl[l])**2)

    MSE_amcl.append(sum(NE_ME[l])/len(df_amcl_clean[l]))

In [20]:
NE_ekf = []
NEk_ekf = 0
ME_ekf = []
NE_ME = []
MSE_ekf = []
x_ekf = []
y_ekf = []

for l in range(5):
    NE_ekf.append([])
    NE_ME.append([])
    x_ekf.append(df_ekf_clean[l][".pose.pose.position.x"])
    y_ekf.append(df_ekf_clean[l][".pose.pose.position.y"])

    for i,j in enumerate(x_truth):
        ex = x_ekf[l][i]-j
        ey = y_ekf[l][i]-y_truth[i]
        NEk = math.sqrt(ex**2+ey**2)
        NEk_ekf = NEk
        NE_ekf[l].append(NEk_ekf)
    
    ME_ekf.append(sum(NE_ekf[l])/len(df_ekf_clean[l]))
    
    for i,j in enumerate(NE_ekf[l]):
        NE_ME[l].append((j-ME_ekf[l])**2)

    MSE_ekf.append(sum(NE_ME[l])/len(df_ekf_clean[l]))

In [34]:
for i in range(5):
    print("{}) mse odom: {:.6f} -- mse ekf imu: {:.6f} -- mse amcl: {:.6f} -- mse ekf: {:.6f}".format(i+1, MSE_odom[i], MSE_imu[i], MSE_amcl[i], MSE_ekf[i]))

1) mse odom: 0.409205 -- mse ekf imu: 0.148651 -- mse amcl: 0.114602 -- mse ekf: 0.148811
2) mse odom: 0.827723 -- mse ekf imu: 0.078898 -- mse amcl: 0.110173 -- mse ekf: 0.078248
3) mse odom: 0.928540 -- mse ekf imu: 0.112603 -- mse amcl: 0.159106 -- mse ekf: 0.114042
4) mse odom: 1.444589 -- mse ekf imu: 0.117744 -- mse amcl: 0.086871 -- mse ekf: 0.119775
5) mse odom: 3.020300 -- mse ekf imu: 0.086941 -- mse amcl: 0.179208 -- mse ekf: 0.089664


In [33]:
def Average(lst):
    return sum(lst) / len(lst)

print("avg mse odom: {:.6f} \navg mse ekf imu: {:.6f} \navg mse amcl: {:.6f} \navg mse ekf: {:.6f}".format(Average(MSE_odom), Average(MSE_imu), Average(MSE_amcl), Average(MSE_ekf)))

avg mse odom: 1.326071 
avg mse ekf imu: 0.108968 
avg mse amcl: 0.129992 
avg mse ekf: 0.110108
